# Purpose

This notebook prepares tract level self-employment data, place of work, broken down by industry / mtc6 sector.

The LEHD WAC files used for travel model taz level data are also place of work and industry, but exclude proprietors.

# Approach
* We need tract level data on self-employed workers - by place of work - broken down by industry.
* That is not available from plain vanilla ACS. The closest is C24070. 	Industry by Class of Worker for the Civilian Employed Population 16 Years and Over. But that is for place of residence geography only.
* Other tables get close - B08528 has the class of worker portion (not industry), but [not available](https://data.census.gov/table/ACSDT5Y2021.B08528?g=1400000US06085500100) at the tract level, though a [place of residence variant is](https://data.census.gov/table/ACSDT5Y2021.B08128?g=1400000US06085500100)

* Instead, we turn to CTPP, which provides a place of work based class of worker accounting in table A202102.
* This is a good start, but it doesn't provide us with key pieces: 
  1. The sectoral breakdown for the self employed workers, which we would need to add to Wage & Salary employment for the TAZDATA.
  1. Temporal currency for the estimates.

For the first one, we rely on county level industry distribution totals to apply to tract level distributions. This is then our "seed" data - a representation of ACS/CTPP 2012-2016 self employed workers, with a sectoral distribution with known deficiencies: It is wrong at the county level insofar as it applies to the total universe of workers not just self employed ones - and it is wrong at the tract level insofar as tracts don't necessarily mirror county distributions.

For the second one, we apply an iterative proportional fitting to scale seed tract data to more current marginals on sectoral and class of worker distributions. We obtain those from ACS PUMS 2019 and 2021, pooling the two years for a larger sample to bring down standard errors and because 2020 is experimental.


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pathlib

# drop = os.environ['DROPBOX_LOC']

/Users/aolsen/opt/anaconda3/envs/geo_env_2/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/1394965940.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Sha

# Mappings and helpers

In [2]:
M_DRIVE = pathlib.Path("/Volumes/Data/Models") if os.name != "nt" else "M:/"

HOME_DIR = pathlib.Path.home()
BOX_DIR = HOME_DIR / 'Library/CloudStorage/Box-Box'

In [3]:
# for halo counties, I rely on the regionalization in http://www.cdss.ca.gov/research/res/pdf/multireports/RegionsofCalifornia.pdf

bayareafips_full = {'06001': 'Alameda', '06013': 'Contra Costa', '06041': 'Marin', '06055': 'Napa',
                    '06075': 'San Francisco', '06081': 'San Mateo', '06085': 'Santa Clara', '06097': 'Sonoma', '06095': 'Solano'}

In [5]:
indus_ctpp_to_mtc = pd.read_excel(
    '/Users/aolsen/Library/CloudStorage/Box-Box/Modeling and Surveys/Regional Modeling/Regional Forecast PBA50 Plus Update/mappings/NAICS_to_ABAG_SECTORS.xlsx', 'ctpp_to_mtc')
indus_ctpp_to_mtc = indus_ctpp_to_mtc.groupby(['CTPP']).MTCname.first()


In [6]:
indus_to_mtc = pd.read_excel(
    '/Users/aolsen/Library/CloudStorage/Box-Box/Modeling and Surveys/Regional Modeling/Regional Forecast PBA50 Plus Update/mappings/NAICS_to_ABAG_SECTORS.xlsx', 'both')

indus_to_mtc['naics_2'] = indus_to_mtc['NAICS-2'].astype(str)
indus_naics2_to_mtc = indus_to_mtc.set_index('naics_2').MTCname


In [8]:
pums_jwtr_map = {1: 'drove_alone',
            2: 'bus/streetcar/trolley',
            3: 'rail',  # 'subway',
            4: 'rail',  # 'railroad',
            5: 'bus/streetcar/trolley',
            6: 'other',
            7: 'other',
            8: 'other',
            9: 'other',
            10: 'other',
            11: 'worked_at_home',
            12: 'other'}

# JWRIP 2-10 provides info on carpool

In [9]:
pums_cow_map = {  # 'b': 'N/A (less than 16 years old/NILF who last worked more than 5 years ago or never worked)',
    1: 'Private',
    2: 'Private',
    3: 'Government',
    4: 'Government',
    5: 'Government',
    6: 'Self',
    7: 'Self',
    8: 'No pay family',
    9: 'Unemployed'}

In [10]:
def simple_estimate_SE(grouped_df, weight='PWGTP', years=1):
    """
    Calculate standard errors, margins of error, and confidence intervals for simple sums using ACS PUMS data.

    Args:
        grouped_grouped_df (pd.GroupBy): A grouped DataFrame containing the ACS PUMS data, typically grouped by relevant variables.
        weight (str): Column name representing replicate weights. Default is 'PWGTP'.
        years (int): Number of years the data represents. Default is 1.

    Returns:
        pd.Series: A Series containing the following statistics:
            - 'Total': Total estimate of the variable.
            - 'ci_upper': Upper bound of the confidence interval.
            - 'ci_lower': Lower bound of the confidence interval.
            - 'se': Standard error.
            - 'moe': Margin of error.
            - 'coef_variation': Coefficient of variation.
            - 'sample_recs': Number of records in the grouped DataFrame.
    """

    # Create a regular expression to match replicate weight columns
    repwgt_str = f'{weight}\d{{1,2}}'

    # Calculate the sum of replicate weights for each column that matches the pattern
    estim_repwgts = grouped_df.filter(regex=repwgt_str).sum().div(float(years))

    # Calculate the sum of primary weights
    estim_prim = grouped_df[weight].sum() / float(years)

    # Calculate squared differences
    squared_diffs = (estim_repwgts - estim_prim)**2
    squared_diffs_summed = squared_diffs.sum()

    # Calculate variance and standard error
    variance = (4 / (80 * years)) * squared_diffs_summed
    standard_error = variance**0.5

    # Calculate coefficient of variation
    coefficient_of_variation = standard_error / estim_prim

    # Calculate margin of error (moe) and confidence intervals
    moe = standard_error * 1.645
    ci_upper = np.ceil(estim_prim + moe)
    ci_lower = np.ceil(estim_prim - moe)

    # Ensure confidence interval lower bound is not negative
    ci_lower = ci_lower if ci_lower > 0 else 0

    # Create a Series with the calculated statistics
    output = pd.Series({
        'Total': int(estim_prim),
        'ci_upper': ci_upper,
        'ci_lower': ci_lower,
        'se': standard_error,
        'moe': moe,
        'coef_variation': coefficient_of_variation,
        'sample_recs': grouped_df.shape[0]
    })

    return output

In [46]:
# data_dict_csv = '/Users/aolsen/Dropbox/My Mac (AOLSEN-MBP.local)/Downloads/PUMS_Data_Dictionary_2022.csv'
# data_dict = pd.read_csv(data_dict_csv, index_col=False, engine='python', names=[
#                         'NAME_OR_VAL', 'VAR', 'DTYPE', 'DWIDTH', 'DESC', 'DET1', 'DET2'])


# cow_map = data_dict.query('NAME_OR_VAL=="VAL" & VAR=="COW"').set_index('DESC').DET2
# cow_map.to_dict()

# Get PUMS data

In [47]:
# 1 year data
PUMS_PATH_2023 = M_DRIVE / 'Data/Census/PUMS/PUMS 2023/psam_p06.csv'
PUMS_PATH_2021 = M_DRIVE / 'Data/Census/PUMS/PUMS 2021/psam_p06.csv'
PUMS_PATH_2019 = M_DRIVE / 'Data/Census/PUMS/PUMS 2019/psam_p06.csv'


In [48]:
KEEP_COLS = ['RT', 'SERIALNO', 'SPORDER', 'PUMA', 'ST', 'ADJINC', 'PWGTP', 'AGEP', 'COW', 'JWTRNS', 'JWRIP', 'ESR', 'MIG',
             'MIGSP', 'SCHL', 'SEX', 'WAGP', 'PINCP', 'JWMNP', 'WKL', 'RAC1P', 'HISP', 'POWPUMA', 'POWSP', 'MIGPUMA', 'POWSP', 'NAICSP']
KEEP_COLS_2023 = ['RT', 'SERIALNO', 'SPORDER', 'PUMA', 'STATE', 'ADJINC', 'PWGTP', 'AGEP', 'COW', 'JWTRNS', 'JWRIP', 'ESR', 'MIG',
             'MIGSP', 'SCHL', 'SEX', 'WAGP', 'PINCP', 'JWMNP', 'WKL', 'RAC1P', 'HISP', 'POWPUMA', 'POWSP', 'MIGPUMA', 'POWSP', 'NAICSP']
REP_WGTS = [f'PWGTP{i}' for i in range(1, 81)]

In [49]:
%%time

pers_data_2023 = pd.read_csv(PUMS_PATH_2023, usecols=KEEP_COLS_2023+REP_WGTS).rename(columns={'STATE':'ST'})

pers_data_2021 = pd.read_csv(PUMS_PATH_2021, usecols=KEEP_COLS+REP_WGTS)

pers_data_2019 = pd.read_csv(PUMS_PATH_2019, usecols=KEEP_COLS+REP_WGTS)

pers_data = pd.concat([pers_data_2019, 
                       pers_data_2021, 
                       pers_data_2023], 
                      keys=[
                      2019, 2021, 2023], 
                      names=['YEAR', 'OID']
                     ).reset_index()

CPU times: user 12.1 s, sys: 2.12 s, total: 14.3 s
Wall time: 14.6 s


## Geographic assignments

In [51]:
pers_data['STCOUNTY'] = pers_data.ST.map(
    lambda x: f'{x:02.0f}')+pers_data.PUMA.map(lambda x: f'{x:05.0f}'[:-2])

pers_data['STPUMA'] = pers_data.ST.apply(
    lambda x: f'{x:02d}') + pers_data.PUMA.apply(lambda x: f'{x:05d}')

In [52]:
# set powpuma var
mask_powpuma = pers_data.POWPUMA.notna()

pers_data.loc[mask_powpuma, 'POWSTPUMA'] = pers_data.loc[mask_powpuma, 'POWSP'].map(
    lambda x: f'{x:03.0f}')+pers_data.loc[mask_powpuma, 'POWPUMA'].map(lambda x: f'{x:05.0f}')

# set powstcounty var, leveraging powsp = 6 and the fact that CA prefixes with county
# though - what about multi-county pumas?
mask_powsp_ca = pers_data.POWSP == 6

pers_data.loc[mask_powsp_ca, 'POWSTCOUNTY'] = pers_data.loc[mask_powsp_ca, 'POWSP'].map(
    lambda x: f'{x:02.0f}')+pers_data.loc[mask_powsp_ca, 'POWPUMA'].map(lambda x: f'{x:05.0f}'[:-2])

pers_data.loc[mask_powsp_ca, 'wrk_county'] = pers_data.loc[mask_powsp_ca,
                                                           'POWSTCOUNTY'].map(bayareafips_full)
pers_data.loc[mask_powsp_ca, 'wrk_county']

6                    NaN
12                   NaN
17                   NaN
19               Alameda
20               Alameda
               ...      
1158463    San Francisco
1158464              NaN
1158465              NaN
1158468     Contra Costa
1158469              NaN
Name: wrk_county, Length: 528692, dtype: object

## Industry and class of worker assignments

In [53]:
pers_data['cow'] = pers_data.COW.map(pums_cow_map).fillna('Not a worker')
pers_data['cow'].value_counts()

cow
Private          487439
Not a worker     470157
Government       109674
Self              82704
Unemployed         6043
No pay family      2453
Name: count, dtype: int64

In [54]:
pers_data['naics_2'] = pers_data.NAICSP.str.slice(
    0, 2).replace({'4M': '44', '3M': '31'})
# pers_data['indus_super'] = pers_data['naics_2'].map(
#     map_naics_supersector).fillna('Not a worker')
# pers_data['indus_super'].dropna()
pers_data['mtc6'] = pers_data.naics_2.map(
    indus_naics2_to_mtc)

In [55]:

# pers_data.loc[(pers_data.naics_2.str.contains('M$',na=False))&(pers_data.naics_2.notna())].NAICSP.unique()

# Prepare PUMS-derived county marginals 

Coefficients of variation are mostly respectable for most observations, but still high for, say, Napa retail where we only have 8 sample records for the estimate.

In [56]:
# place of work based accounting of self employed workes by industry

cow_indus_summary_pow_2023 = pers_data.query('POWSTCOUNTY.isin(@bayareafips_full)  & YEAR==2023').groupby(
    ['wrk_county',  'cow',  'mtc6']).apply(simple_estimate_SE, years=1)

cow_indus_summary_pow_2023.Total.loc(0)['San Francisco', 'Self']
cow_indus_summary_pow_2023

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2493765978.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cow_indus_summary_pow_2023 = pers_data.query('POWSTCOUNTY.isin(@bayareafips_full)  & YEAR==2023').groupby(


Total  ci_upper  ci_lower           se  \
wrk_county cow        mtc6                                                
Alameda    Government agrempn    234.0     567.0       0.0   201.971038   
                      fpsempn  10288.0   12153.0    8424.0  1133.156057   
                      herempn  60619.0   65028.0   56211.0  2680.073506   
                      mwtempn   9766.0   11689.0    7844.0  1168.496128   
                      othempn  35703.0   39589.0   31818.0  2362.307071   
...                                ...       ...       ...          ...   
Sonoma     Self       fpsempn  11256.0   13329.0    9184.0  1259.904560   
                      herempn  12263.0   14547.0    9980.0  1388.320550   
                      mwtempn   3156.0    4175.0    2138.0   619.349901   
                      othempn   4022.0    5436.0    2609.0   859.498284   
                      retempn   3233.0    4529.0    1938.0   787.710829   

                                       moe  coef_variation  sample_recs  
wrk_county cow        mtc6                                               
Alameda    Government agrempn   332.242357        0.863124          2.0  
                      fpsempn  1864.041714        0.110143        113.0  
                      herempn  4408.720918        0.044212        662.0  
                      mwtempn  1922.176130        0.119649        103.0  
                      othempn  3885.995133        0.066166        344.0  
...                                    ...             ...          ...  
Sonoma     Self       fpsempn  2072.543001        0.111932        129.0  
                      herempn  2283.787305        0.113212        139.0  
                      mwtempn  1018.830587        0.196245         42.0  
                      othempn  1413.874677        0.213699         48.0  
                      retempn  1295.784314        0.243647         28.0  

[183 rows x 7 columns]

In [57]:
# we approximate 2020 by averaging 2019 and 2021
cow_indus_summary_pow = pers_data.query('POWSTCOUNTY.isin(@bayareafips_full) & YEAR.isin([2019,2021])').groupby(
    ['wrk_county',  'cow', 'mtc6']).apply(simple_estimate_SE, years=2)

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/3827224026.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cow_indus_summary_pow = pers_data.query('POWSTCOUNTY.isin(@bayareafips_full) & YEAR.isin([2019,2021])').groupby(


In [59]:
county_industry_marginals_2020 = cow_indus_summary_pow.Total.loc(0)[:,'Self'].reset_index(
    1).Total.unstack().fillna(0).astype(int)#.to_records()


county_industry_marginals_2020.index = county_industry_marginals_2020.index.set_names('county_name')
county_industry_marginals_2020.columns = county_industry_marginals_2020.columns.set_names('industry')
county_industry_marginals_2020

industry,agrempn,fpsempn,herempn,mwtempn,othempn,retempn
county_name,,,,,,
Alameda,640,25920,29133,11251,10114,4719
Contra Costa,849,19891,15662,3809,6230,2889
Marin,338,9496,7400,1652,2843,1271
Napa,427,1882,2565,952,1266,403
San Francisco,0,24105,19800,7751,8352,4281
San Mateo,281,14022,11067,4142,5413,1736
Santa Clara,253,31829,24836,10019,10870,4476
Solano,256,4263,5152,1707,2286,1162
Sonoma,932,9414,10761,2761,5262,2696


In [60]:
county_industry_marginals_2023 = cow_indus_summary_pow_2023.Total.loc(0)[:,'Self'].reset_index(
    1).Total.unstack().fillna(0).astype(int)#.to_records()


county_industry_marginals_2023.index = county_industry_marginals_2023.index.set_names('county_name')
county_industry_marginals_2023.columns = county_industry_marginals_2023.columns.set_names('industry')
county_industry_marginals_2023

industry,agrempn,fpsempn,herempn,mwtempn,othempn,retempn
county_name,,,,,,
Alameda,502,26786,25500,9102,12959,4365
Contra Costa,249,19176,18067,5683,8412,2507
Marin,91,8248,6805,697,1985,1490
Napa,778,3623,3438,2310,792,396
San Francisco,381,28511,19938,7057,7537,2564
San Mateo,400,16178,11290,3922,4963,1898
Santa Clara,615,26127,29561,10242,12062,2887
Solano,0,2698,5960,905,2835,1098
Sonoma,479,11256,12263,3156,4022,3233


# CTPP data

In [61]:
# ctpp_cow_codes = {1: 'Total, all class of worker',
#              2: 'Private for-profit wage and salary workers',
#              3: 'Private not-for-profit wage and salary workers',
#              4: 'Local government workers',indus
#              5: 'State government workers',
#              6: 'Federal government workers',
#              7: 'Self-employed workers in own not incorporated business',
#              8: 'Self-employed workers in own, incorporated business',
#              9: 'Unpaid family workers'}

ctpp_cow_codes = {#1: 'Total, all class of worker',
             2: 'Private',
             3: 'Private',
             4: 'Government',
             5: 'Government',
             6: 'Government',
             7: 'Self-employed',
             8: 'Self-employed',
             9: 'Unpaid family workers'}

In [62]:
data_dict_all = pd.read_excel(
    M_DRIVE / 'Data/Census/CTPP/CTPP2012-2016/2012-2016 CTPP documentation/2012-2016 CTPP Final Table Specs.xlsx', 'Table Specs', skiprows=[1])

# detailed / itemized
ctpp_indus_codes = data_dict_all.query('`Table ID`=="A202212" & Type=="D" ').set_index([
    'Line Number']).Stub

# summary level totals
ctpp_indus_codes = data_dict_all.query('`Table ID`=="A202212" & Type=="I" ').set_index([
    'Line Number']).Stub
ctpp_indus_codes

Line Number
2.0     Agriculture, forestry, fishing and hunting, an...
3.0                                          Construction
4.0                                         Manufacturing
5.0                                       Wholesale trade
6.0                                          Retail trade
7.0         Transportation and warehousing, and utilities
8.0                                           Information
9.0     Finance, insurance, real estate and rental and...
10.0    Professional, scientific, management, administ...
11.0              Educational, health and social services
12.0    Arts, entertainment, recreation, accommodation...
13.0        Other services (except public administration)
14.0                                Public administration
15.0                                         Armed forces
Name: Stub, dtype: object

In [63]:
ctpp_indus_cow_all_codes = data_dict_all.query(
    '`Table ID`=="A202220" & `Line Number`>0').set_index(['Line Number']).Stub  # .to_dict()
ctpp_indus_cow_part_codes = data_dict_all.query(
    '`Table ID`=="A202220" & 90<`Line Number`<120').set_index(['Line Number']).Stub  # .to_dict()
ctpp_indus_cow_all_codes

Line Number
1.0                                                  Total
2.0      Agriculture, forestry, fishing and hunting, an...
3.0                                           Construction
4.0                                          Manufacturing
5.0                                        Wholesale trade
                               ...                        
131.0              Educational, health and social services
132.0    Arts, entertainment, recreation, accommodation...
133.0        Other services (except public administration)
134.0                                Public administration
135.0                                         Armed forces
Name: Stub, Length: 135, dtype: object

In [64]:
CTPP_BASE_PATH = M_DRIVE / 'Data/Census/CTPP/CTPP2012-2016/ca/06'

In [65]:
data_cow_indus = pd.read_csv(
    CTPP_BASE_PATH / 'CA_2012thru2016_A202220.csv')
data_cow_indus['SUMLEVEL'] = data_cow_indus.GEOID.str.slice(0, 3)
data_cow_indus.head()

,GEOID,TBLID,LINENO,EST,MOE,SOURCE,SUMLEVEL
0,C2200US06,A202220,1,"17,192,045","+/-20,898",NaN,C22
1,C2200US06,A202220,2,"399,070","+/-5,797",NaN,C22
2,C2200US06,A202220,3,"1,024,990","+/-7,841",NaN,C22
3,C2200US06,A202220,4,"1,657,620","+/-10,586",NaN,C22
4,C2200US06,A202220,5,"519,670","+/-5,831",NaN,C22


In [66]:
def process_CTTP_data(table_id='A202102', sumlevel='C31'):
    """
    Process data from CTPP, subsetted to the Bay Area.

    Loads data from a CSV file, subsets it to California (CA), and then further
    narrows it down to the Bay Area. It also extracts relevant columns and performs
    data transformations.

    Returns:
    cow_data_tract_bayarea: A DataFrame containing the processed data for the Bay Area tracts.
    """

    # Load the CTPP data
    file_path = CTPP_BASE_PATH / f'CA_2012thru2016_{table_id}.csv'
    print(file_path)

    data = pd.read_csv(
        file_path)
    data['SUMLEVEL'] = data.GEOID.str.slice(0, 3)

    # Subset data to relevant summary level
    data_tract = data[data.GEOID.str.slice(0, 3) == sumlevel]

    # Get the numeric value, stripping formating characters
    data_tract['value'] = pd.to_numeric(data_tract.EST.str.replace(',', ''))

    # Extract county name, geoid10
    data_tract['county_name'] = data_tract.GEOID.str.slice(
        7, 12).map(bayareafips_full)

    # this will be tract level detail when C31 is passed as sumlevel
    data_tract['geoid10'] = data_tract.GEOID.str.slice(7, 18)

    # Subset to Bay Area tracts
    cow_data_tract_bayarea = data_tract[data_tract.county_name.isin(
        bayareafips_full.values())]

    return cow_data_tract_bayarea

## Tract level data for industry

(We don't currently use this directly, but we roll up county level data from these)

In [67]:
indus_data_tract_bayarea = process_CTTP_data(
    table_id='A202212', sumlevel='C31')
indus_data_tract_bayarea.head()

/Volumes/Data/Models/Data/Census/CTPP/CTPP2012-2016/ca/06/CA_2012thru2016_A202212.csv


/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2792952791.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2792952791.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tract['value'] = pd.to_numeric(data_tract.EST.str.replace(',', ''))
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2792952791.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

,GEOID,TBLID,LINENO,EST,MOE,SOURCE,SUMLEVEL,value,county_name,geoid10
174720,C3100US06001400100,A202212,1,"1,045",+/-197,NaN,C31,1045,Alameda,06001400100
174721,C3100US06001400100,A202212,2,10,+/-15,NaN,C31,10,Alameda,06001400100
174722,C3100US06001400100,A202212,3,65,+/-44,NaN,C31,65,Alameda,06001400100
174723,C3100US06001400100,A202212,4,50,+/-42,NaN,C31,50,Alameda,06001400100
174724,C3100US06001400100,A202212,5,25,+/-35,NaN,C31,25,Alameda,06001400100


In [68]:
# code industries based on data dict values (to NAICS-2), and then transcode to MTC-6

indus_data_tract_bayarea['industry'] = indus_data_tract_bayarea.LINENO.map(
    ctpp_indus_codes.map(indus_ctpp_to_mtc))

# get just the high level industry total numbers from the A202212 table
# this means we won't inadvertently double count across summary levels
indus_linenos = list(range(2, 16))

indus_data_tract_bayarea = indus_data_tract_bayarea.query(
    'LINENO.isin(@indus_linenos)')

indus_data_tract_bayarea = indus_data_tract_bayarea.groupby(
    ['geoid10', 'county_name', 'industry']).value.sum().unstack('industry').fillna(0).astype(int)

In [69]:
indus_data_tract_bayarea.groupby(level=1).sum()#.sum(axis=1)

industry,agrempn,fpsempn,herempn,mwtempn,othempn,retempn
county_name,,,,,,
Alameda,1581,119722,214354,118660,67458,56300
Contra Costa,3211,77397,117949,40581,36726,34529
Marin,681,28290,41494,10730,13808,12645
Napa,3471,8280,26780,11935,6250,6563
San Francisco,905,192937,189078,50652,77419,50117
San Mateo,1342,88028,95053,62402,37768,31170
Santa Clara,4100,208746,261354,201847,106190,79444
Solano,1778,15705,43800,17916,21213,16441
Sonoma,5199,32357,72124,32241,21638,23652


## Tract level data for class of workers

In [70]:
cow_data_tract_bayarea = process_CTTP_data(table_id='A202102', sumlevel='C31')
cow_data_tract_bayarea['class_of_worker'] = cow_data_tract_bayarea.LINENO.map(
    ctpp_cow_codes)
cow_data_tract_bayarea = cow_data_tract_bayarea.query('LINENO!=1')
cow_data_tract_bayarea = cow_data_tract_bayarea.groupby(
    ['geoid10', 'county_name', 'class_of_worker']).value.sum().unstack('class_of_worker').fillna(0).astype(int)

/Volumes/Data/Models/Data/Census/CTPP/CTPP2012-2016/ca/06/CA_2012thru2016_A202102.csv


/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2792952791.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2792952791.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tract['value'] = pd.to_numeric(data_tract.EST.str.replace(',', ''))
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_64203/2792952791.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [71]:
cow_data_tract_bayarea['Self-employed'].sum()

329196

### County level data for class of workers

In [72]:
cow_data_county_bayarea = cow_data_tract_bayarea.groupby(level='county_name').sum()
cow_data_county_bayarea.sum(axis=1)

county_name
Alameda          578058
Contra Costa     310447
Marin            107752
Napa              63259
San Francisco    560973
San Mateo        315787
Santa Clara      861788
Solano           116938
Sonoma           187240
dtype: int64

## County level data for industry

In [73]:
# indus_data_county_bayarea = process_CTTP_data(
#     table_id='A202212', sumlevel='C31')
# indus_data_county_bayarea['industry'] = indus_data_county_bayarea.LINENO.map(
#     ctpp_indus_codes.map(indus_ctpp_to_mtc))

# # get just the high level industry total numbers from the A202212 table
# indus_data_county_bayarea = indus_data_county_bayarea.query('LINENO!=1')


indus_data_county_bayarea = indus_data_tract_bayarea.groupby(level=1).sum()

In [74]:
indus_data_county_bayarea_pct = indus_data_county_bayarea.div(
    indus_data_county_bayarea.sum(axis=1), axis=0)
indus_data_county_bayarea_pct

industry,agrempn,fpsempn,herempn,mwtempn,othempn,retempn
county_name,,,,,,
Alameda,0.002735,0.207105,0.370807,0.205267,0.116694,0.097392
Contra Costa,0.010345,0.249352,0.379999,0.130741,0.118321,0.111243
Marin,0.006326,0.262801,0.385460,0.099677,0.128270,0.117466
Napa,0.054852,0.130849,0.423205,0.188609,0.098769,0.103715
San Francisco,0.001613,0.343850,0.336973,0.090271,0.137975,0.089318
San Mateo,0.004250,0.278779,0.301026,0.197623,0.119609,0.098713
Santa Clara,0.004758,0.242254,0.303307,0.234248,0.123236,0.092197
Solano,0.015216,0.134400,0.374830,0.153321,0.181536,0.140698
Sonoma,0.027771,0.172837,0.385255,0.172217,0.115581,0.126339


In [75]:
# we now have data on county level industry distribution as well as tract level counts of self emp workers
# we use their product as the seed distribution

tract_seed_self_employed = indus_data_county_bayarea_pct.mul(
    cow_data_tract_bayarea['Self-employed'], axis=0)
# tract_seed_self_employed.columns.set_names('class_detail')

In [76]:
cow_data_tract_bayarea['Self-employed'].sum()

329196

In [77]:
tract_seed_self_employed.sum(axis=1).groupby('county_name').sum()

county_name
Alameda          59973.0
Contra Costa     39935.0
Marin            19240.0
Napa              7277.0
San Francisco    58773.0
San Mateo        33477.0
Santa Clara      71033.0
Solano           11018.0
Sonoma           28470.0
dtype: float64

In [78]:
tract_seed_self_employed.sum()
#indus_data_county_bayarea.sum()/indus_data_county_bayarea.sum().sum()

industry
agrempn      2631.362249
fpsempn     81538.336368
herempn    114434.807793
mwtempn     55974.921958
othempn     41068.221705
retempn     33548.349928
dtype: float64

# Update 2012-2016 data to PUMS derived marginals data using IPF

In [79]:
from ipfn import ipfn


def perform_ipf(seed_data, marginal_data, dimensions):
    """
    Quick wrapper for Iterative Proportional Fitting (IPF) on the seed data.

    Parameters:
        seed_data (DataFrame): The initial seed data in long format.
        marginal_data (DataFrame): The marginal totals data, indexed by the same dimensions as `seed_data`.
        dimensions (list of list): A list of lists where each sublist specifies the dimensions for IPF aggregation.

    Returns:
        DataFrame: The updated seed data after IPF adjustment.
    """


    original_total = seed_data['total'].sum()
    print('Before IPF adjustment:', original_total)

    # Prepare marginal data
    margins_data_long = marginal_data.stack()

    # Denote marginals and their mappings
    aggregates = [margins_data_long]

    # Perform IPF
    IPF = ipfn.ipfn(seed_data, aggregates, dimensions)
    updated_data = IPF.iteration()
    updated_total = updated_data['total'].sum()
    print('After IPF adjustment:', updated_total)

    return updated_data


dimensions = [['county_name', 'industry']]

In [88]:
tract_seed_self_employed_long = tract_seed_self_employed.stack().reset_index(name='total')

tract_data_updated_2020 = perform_ipf(
    seed_data=tract_seed_self_employed_long,
    marginal_data=county_industry_marginals_2020,
    dimensions=dimensions
)

tract_data_updated_2020['tract10'] = tract_data_updated_2020.geoid10
tract_data_updated_2020.total = tract_data_updated_2020.total.round(0).astype(int)

Before IPF adjustment: 329196.0
After IPF adjustment: 391487.0


In [89]:
tract_seed_self_employed_long = tract_seed_self_employed.stack().reset_index(name='total')

tract_data_updated_2023 = perform_ipf(
    seed_data=tract_seed_self_employed_long,
    marginal_data=county_industry_marginals_2023,
    dimensions=dimensions
)
tract_data_updated_2023['tract10'] = tract_data_updated_2023.geoid10
tract_data_updated_2023.total = tract_data_updated_2023.total.round(0).astype(int)

Before IPF adjustment: 329196.0
After IPF adjustment: 397999.0


## Check against marginals
They should match to a 't' for the relevant dimension
(`self_emp_combo.upd` vs `margins_data_long`)

checks out before rounding - after rounding, small loss of precision


In [90]:
# small rounding discrepancy
print(tract_data_updated_2023.groupby('county_name').total.sum().div(
    county_industry_marginals_2023.sum(axis=1)))

county_name
Alameda          0.999912
Contra Costa     0.999704
Marin            1.000052
Napa             0.999559
San Francisco    0.999955
San Mateo        0.999715
Santa Clara      1.000000
Solano           0.997999
Sonoma           1.000407
dtype: float64


In [91]:
# small rounding discrepancy
print(tract_data_updated_2023.groupby('industry').total.sum().div(county_industry_marginals_2023.sum(axis=0)))

industry
agrempn    0.997997
fpsempn    0.999825
herempn    1.000241
mwtempn    0.999837
othempn    0.999568
retempn    0.998875
dtype: float64


In [92]:
# # write to disk
# out_path = '.'

# out_df.to_csv(os.path.join(out_path,'tract_self_employed_workers_2020.csv'))

# Translate TRACT to TAZ geographies

We need the data by TAZ geographies. 

* We assign census 2010 blocks to the containing TAZ based on block centroid.
* We then use WAC jobs data as weights (instead of just area, to get a "behavioral" datum) - summarizing blocks by tract and taz and getting for each tract the share of jobs in related TAZs.

In [93]:
# Get TAZ zones

ZONE_PATH = BOX_DIR /  'Modeling and Surveys/Urban Modeling/Spatial/Zones/TAZ1454/zones1454.shp'

zones = gpd.read_file(
    ZONE_PATH).to_crs('EPSG:26910')

In [94]:
import geopandas as gpd
from pygris import blocks


def fetch_bayarea_blocks(output_path, year):

    if not os.path.exists(output_path):
        marin_blocks = blocks(state='CA', county='041', year=year, cache=True)
        napa_blocks = blocks(state='CA', county='055', year=year, cache=True)
        solano_blocks = blocks(state='CA', county='095', year=year, cache=True)
        sonoma_blocks = blocks(state='CA', county='097', year=year, cache=True)
        alameda_blocks = blocks(
            state='CA', county='001', year=year, cache=True)
        contracosta_blocks = blocks(
            state='CA', county='013', year=year, cache=True)
        sanmateo_blocks = blocks(
            state='CA', county='081', year=year, cache=True)
        santaclara_blocks = blocks(
            state='CA', county='085', year=year, cache=True)
        sanfrancisco_blocks = blocks(
            state='CA', county='075', year=year, cache=True)

        bayarea_blocks = pd.concat([marin_blocks, napa_blocks, solano_blocks, sonoma_blocks,
                                    sanmateo_blocks, santaclara_blocks, sanfrancisco_blocks,
                                    contracosta_blocks, alameda_blocks])

        bayarea_blocks.to_feather(output_path)
    else:
        bayarea_blocks = gpd.read_feather(output_path)

    bayarea_blocks = bayarea_blocks.to_crs('EPSG:26910')
    return bayarea_blocks


output_path = M_DRIVE / 'Data/Census/GIS/blocks/bayarea_blocks_{year}.feather'
#output_path = '/Users/aolsen/Downloads/bayarea_blocks_{year}.feather'

bayarea_blocks = fetch_bayarea_blocks(output_path, 2010)

In [95]:
# Wac data, LODES 7 geo vintage (2010s)
# Nov 2024, AO: Why did I pick 2014 specifically when I did this?

WAC_PATH = M_DRIVE / 'Data/Census/LEHD/Workplace Area Characteristics (WAC)/ca_wac_S000_JT00_2014.csv'

wac_2014 = pd.read_csv(
    WAC_PATH, dtype={'w_geocode': str})
wac_2014['county_name'] = wac_2014.w_geocode.str.slice(
    0, 5).map(bayareafips_full)
wac_2014 = wac_2014[wac_2014.county_name.notna()]

# bay area total jobs, by place of work
wac_2014.C000.sum()

3565696

In [97]:
def pct(x): return x/x.sum()

# prepare a tract-to-zone weighting series

# assign to census blocks file to use for weighting tracts to TAZ

bayarea_blocks['jobs'] = bayarea_blocks.GEOID10.map(
    wac_2014.set_index('w_geocode').C000)


# get tract10 identifier from block id
bayarea_blocks['tract10'] = bayarea_blocks.GEOID10.str.slice(0, 11)


# get centroid / representative point for block
bayarea_blocks['geom_pt'] = bayarea_blocks.representative_point()

# for each block, get the zone it falls within

blocks_x_zones = gpd.sjoin(bayarea_blocks.set_geometry('geom_pt'), zones)


# sum BLOCK level jobs into both ZONES and TRACTS so we can figure out the share of each tract that goes to each zone
# (this is needed to map tract level employment data on self employment back to TAZs)


jobs_by_tract_zone = blocks_x_zones.groupby(['tract10', 'zone_id']).jobs.sum()
jobs_by_tract_zone_pct = jobs_by_tract_zone.groupby(
    level='tract10', group_keys=False).apply(pct)


# The resulting weighting series, for multiplication with a tract10-indexed series
jobs_by_tract_zone_pct.head()

tract10      zone_id
06001400100  914        0.002128
             1005       0.997872
             1007       0.000000
             1028       0.000000
             1155       0.000000
Name: jobs, dtype: float64

In [98]:
# self_emp_series = out_df.set_index(
#     ['tract10', 'industry']).value  # .unstack('industry')
# self_emp_series.sum()

## 2023 version
From Nov 2023

In [100]:
out_path = '.'

In [101]:
self_emp_series_2023 = tract_data_updated_2023.set_index(
    ['tract10', 'industry']).total
self_emp_series_2023_distributed = self_emp_series_2023.mul(
    jobs_by_tract_zone_pct)

# sum to TAZs

self_emp_distributed_2023_out = self_emp_series_2023_distributed.groupby(
    level=['zone_id', 'industry']).sum().round(0).astype(int).reset_index(name='value')

self_emp_distributed_2023_out.to_csv(os.path.join(
    out_path, 'taz_self_employed_workers_2023.csv'))
self_emp_distributed_2023_out.groupby('industry').value.sum()

industry
agrempn      3485
fpsempn    142577
herempn    132861
mwtempn     43060
othempn     55548
retempn     20424
Name: value, dtype: int64

## 2020 version

In [102]:
self_emp_series_2020 = tract_data_updated_2020.set_index(
    ['tract10', 'industry']).total
self_emp_series_2020_distributed = self_emp_series_2020.mul(
    jobs_by_tract_zone_pct)

# sum to TAZs

self_emp_distributed_2020_out = self_emp_series_2020_distributed.groupby(
    level=['zone_id', 'industry']).sum().round(0).astype(int).reset_index(name='value')


self_emp_distributed_2020_out.to_csv(os.path.join(
    out_path, 'taz_self_employed_workers_2020.csv'))
self_emp_distributed_2020_out.groupby('industry').value.sum()

industry
agrempn      3940
fpsempn    140831
herempn    126352
mwtempn     44058
othempn     52636
retempn     23653
Name: value, dtype: int64

# Sandbox -
## Checking 2019, 2021 SF non-work from home employment.

In [121]:
pers_data.query('POWSTCOUNTY.isin(@bayareafips_full)').groupby(
    ['wrk_county',  'cow', 'mtc6']).apply(simple_estimate_SE, years=2)

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_97041/3590922956.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pers_data.query('POWSTCOUNTY.isin(@bayareafips_full)').groupby(


Total  ci_upper  ci_lower           se  \
wrk_county cow        mtc6                                                
Alameda    Government agrempn     44.0      97.0       0.0    31.584510   
                      fpsempn  10746.0   11718.0    9775.0   590.295090   
                      herempn  63252.0   65732.0   60773.0  1507.589923   
                      mwtempn  10161.0   11197.0    9126.0   629.442417   
                      othempn  35595.0   37507.0   33684.0  1162.248446   
...                                ...       ...       ...          ...   
Sonoma     Self       fpsempn   9414.0   10402.0    8427.0   600.503971   
                      herempn  10761.0   11782.0    9742.0   620.273826   
                      mwtempn   2761.0    3445.0    2078.0   415.504941   
                      othempn   5262.0    5955.0    4571.0   420.634862   
                      retempn   2696.0    3180.0    2213.0   294.134759   

                                       moe  coef_variation  sample_recs  
wrk_county cow        mtc6                                               
Alameda    Government agrempn    51.956518        0.709764          1.0  
                      fpsempn   971.035424        0.054932        226.0  
                      herempn  2479.985423        0.023835       1334.0  
                      mwtempn  1035.432776        0.061947        194.0  
                      othempn  1911.898694        0.032652        726.0  
...                                    ...             ...          ...  
Sonoma     Self       fpsempn   987.829032        0.063788        222.0  
                      herempn  1020.350443        0.057638        221.0  
                      mwtempn   683.505628        0.150491         56.0  
                      othempn   691.944349        0.079931        101.0  
                      retempn   483.851678        0.109100         59.0  

[199 rows x 7 columns]